# Virrevandring

> Engelsk: Random walks

Virrevandring er en prosess hvor vi følger et objekt som beveger seg tilfeldig.

**Jeg har skrevet litt plottekode fra før:**

In [ ]:
%matplotlib inline
from IPython.display import HTML
import matplotlib.pyplot as plt
import numpy as np

import  animate

plt.rc('axes', labelsize=14) 
plt.rc('legend', fontsize=10)

## Hva brukes virrevandring til?

Brukes mye i vitenskapelige anvendelser:

- Modeller for aksjekurs
- Populasjonsmodeller
- Modeller for genetikk (genetisk drift)
- Studier av polymerer i materialvitenskap
- Segmentering av bilder
- Twitter bruker virrevandring til å foreslå nye personer du kan følge


Vi skal bruke **en partikkel som har en viss sannsynlighet for å bevege seg en gitt vei**.

##  Én dimensjon

Simulering av gange langs en linje

$$
    x_{n + 1} = x_{n } + \Delta x_{n + 1},
$$

gitt et utgangspunkt $x_0$ og *stokastisk* steglengde $\Delta x_{n + 1}$.

![](figures/1D_walk.png)

### Steglengden $\Delta x_n$

Steglengden er en **stokastisk variabel** med en gitt distribusjon. 

Bruker ofte normalfordeling eller uniform fordeling.

**Implementasjon:** Vi skal bruke 

$$
    x_0 = 0
$$ 

og 

$$
    \Delta x_{n + 1} = \pm 1.
$$

Det er lik sannsynlighet for å gå begge veier.

In [ ]:
N = 10
dx = [-1, 1]
x = np.zeros(N)
...
print(x)

En enkel implementasjon kan f.eks bruke `numpy.random.choice`:

In [ ]:
N = 10
dx = [-1, 1]
x = np.zeros(N)
for n in range(N):
    x[n] = x[n - 1] + np.random.choice(dx)
print(x)

**Vektorisert implementasjon.** 

Vi kan bruke `np.cumsum` til å beregne den kumulative summen:


$$
    x_{n} = \sum_{i=1}^{n} \Delta x_i.
$$


In [ ]:
def random_walk_1D(N):
    x = np.zeros(N)
    ...
    return x

Implementasjonen kan da se slik ut:

In [ ]:
def random_walk_1D(N):
    dx = np.random.choice([-1, 1], size=(N-1))
    x = np.zeros(N)
    x[1:] = np.cumsum(dx)
    return x

**Plotting av virrevandring i 1D:**

In [ ]:
N = 50
x = random_walk_1D(N)

fig = plt.figure(figsize=(15, 5))
plt.step(range(N), x, where="mid", color="black")
plt.xlabel("n")
plt.ylabel("x")
plt.show()
print()

**Veien vil endre seg hver gang du kjører programmet:**

In [ ]:
fig = plt.figure(figsize=(15, 5))
N = 50
for _ in range(10):
    plt.step(range(N), random_walk_1D(N), where="mid", alpha=0.75)
plt.xlabel("n")
plt.ylabel("x")
plt.show()
print()

### Forventnigsverdier

Det generelle utrykket for posisjonen er

$$
    x_{n} = x_0 + \sum_{i=1}^{n} \Delta x_i,
$$

men vi forenkler og bruker $x_0 = 0$.

*Forventet posisjon* blir da det samme som *forventet forflytning*.

**Forventningsverdi for posisjon** er gitt ved

$$
    \langle x_{n}\rangle = \sum_{i=1}^{n} \langle\Delta x_i \rangle,
$$

siden $\langle a + b\rangle = \langle a\rangle + \langle b\rangle$.

Forventningsverdien til steglengden blir

$$
     \langle\Delta x_i \rangle = \frac{1}{2} \cdot 1 + \frac{1}{2} \cdot (-1) = 0 .
$$


Forventet gjennomsnittlig forflytning er derfor

$$
     \langle x_{n}\rangle = 0 .
$$


**Bruk av mange vandrere** vil være symmetrisk rundt $x_0$.

In [ ]:
N = 500
walkers = 1000
n_1D = np.arange(N)
X = np.zeros((walkers, N))
plt.figure(figsize=(15, 5))
for k in range(walkers):
    X[k] = random_walk_1D(N)
    plt.step(n_1D, X[k], where="mid", color="black", alpha=0.01)
plt.xlabel("n")
plt.ylabel("x")
plt.show()
print()

**Forventningsverdi til den kvadrerte posisjonen** er gitt ved

$$
    \langle x_{n}^2 \rangle 
    =  \langle \left(\sum_{i=1}^{n} \Delta x_i \right)^2 \rangle 
    = \sum_{i=1}^{n} \langle \Delta x_i^2 \rangle
    + 2\sum_{i < j}\langle\Delta x_i \cdot \Delta x_j \rangle .
$$


Ettersom $\Delta x_i$ og $\Delta x_j$ er uavhengige har vi at

$$
    \langle\Delta x_i \cdot \Delta x_j \rangle 
    = \langle\Delta x_i\rangle \cdot \langle\Delta x_j\rangle = 0,
$$

hvor vi nettopp så at $\langle\Delta x_i\rangle = 0$.

Vi kan regne ut forventingsverdien til den kvadrerte steglengden:

$$
    \langle \Delta x_i^2 \rangle = \frac{1}{2}(1)^2 + \frac{1}{2}(-1)^2 = 1 .
$$


Forventningsverdien til den kvadrerte forflytningen er da

$$
    \langle x_{n}^2 \rangle = n .
$$


**Variansen** er  gitt ved

$$
    \sigma^2(x_{n}) = \langle x_{n}^2 \rangle  - \langle x_{n}\rangle^2 = n .
$$


**Standardavviket** er

$$
    \sigma(x_{n}) = \sqrt{\sigma^2(x_{n})} = \sqrt{n} .
$$


### De store talls lov

Når populasjonen øker, vil snittet gå mot forventningsverdien.

In [ ]:
plt.figure(figsize=(15, 5))
ax1 = plt.subplot(121, xlabel="n", ylabel=r'$\sigma$')
ax2 = plt.subplot(122, xlabel="n", ylabel=r'$\langle x \rangle$')
ax1.plot(n_1D, np.sqrt(n_1D), label="Analytic", color="black")
ax2.plot(n_1D, np.zeros(len(n_1D)), color="black")
for w in [10, 100, 1000]:
    sigma = np.std(X[:w], axis=0)
    ax1.plot(n_1D, sigma, "--", label=f"{w} walkers")
    ax2.plot(n_1D, np.mean(X[:w], axis=0), "--")
ax1.legend()
plt.show()
print()

## To dimensjoner

Generelt er formelen

$$
    \vec{r}_{n} = \vec{r}_0 + \sum_{i = 1}^n \Delta \vec{r}_i 
$$

for en gitt startposisjon $\vec{r}_0$.

I to dimensjoner har vi

$$
    \vec{r}_n = (x_n, y_n).
$$


### Steglengden $\Delta \vec{r}_n$

Steglengden er en stokastisk variabel i to dimensjoner

$$
    \Delta \vec{r}_n = (\Delta x_n, \Delta y_n),
$$

hvor distribusjonen kan være den samme eller forskjellig i $x$- og $y$-retningen. 

**Implementasjon** Vi skal bruke 

$$
    \vec{r}_0 = (0, 0) 
$$  

og 

$$
    \Delta \vec{r}_n  \in \{(1, 0), (-1, 0), (0, 1), (0, -1)\} .
$$ 


Altså kan vandreren vår gå et steg mot nord, sør, øst eller vest.

In [ ]:
def random_walk_2D(N):
    dr = np.array([(1, 0), (-1, 0), (0, 1), (0, -1)])
    r = np.zeros((N, 2))
    ...
    return r

Implementasjonen kan da se slik ut:

In [ ]:
def random_walk_2D(N):
    dr = np.array([(1, 0), (-1, 0), (0, 1), (0, -1)])
    idx = np.random.randint(4, size=(N-1))
    r = np.zeros((N, 2))
    r[1:] = np.cumsum(dr[idx], axis=0)
    return r

**Videosnutt av virrevandring i 2D:**

In [ ]:
r = random_walk_2D(50)
video = animate.RandomWalk(r, figsize=(10, 10)).video(speed=200)
HTML(video)

### Forventningsverdier

Det generelle utrykket for posisjon er gitt ved

$$
    \vec{r}_{n} =  \vec{r}_0 + \sum_{i=1}^{n} \Delta \vec{r}_i,
$$

men vi har satt $\vec{r}_0 = (0, 0)$. 

Forventet posisjon og forventet forflytning er igjen det samme.

Forventningsverdien til posisjonen er gitt ved

$$
    \langle \vec{r}_{n} \rangle 
    = (\langle x_{n}\rangle, \langle y_{n}\rangle) .
$$

Forventningsverdien til $x$ er gitt ved

$$
    \langle x_{n}\rangle = \sum_{i=1}^{n} \langle\Delta x_i \rangle, 
$$


hvor forventningsverdien til $\Delta x$ er 

$$
     \langle\Delta x_i \rangle = \frac{1}{4} \cdot 1 + \frac{1}{4} \cdot (-1) + \frac{1}{2} \cdot 0 = 0.
$$


Vi får da at $\langle x_{n}\rangle = \langle y_{n}\rangle = 0$, som gir

$$
   \langle \vec{r}_{n}\rangle = (0, 0).
$$

**Bruk av mange vandrere** vil være symmetrisk rundt $\vec{r}_0$.

In [ ]:
video = animate.RandomWalk([random_walk_2D(50) for _ in range(10)]).video(speed=150)
HTML(video)

**Forventet kvadrert avstand fra startpunkt** er gitt ved

$$
    \langle |\vec{r}_{n}|^2 \rangle 
    = \langle x_{n}^2 \rangle + \langle y_{n}^2 \rangle .
$$


Tilsvarende som i én dimensjon får vi

$$
    \langle x_{n}^2 \rangle 
    = \sum_{i=1}^{n} \langle \Delta x_i^2 \rangle
    + 2\sum_{i < j}\langle\Delta x_i\rangle \cdot \langle\Delta x_j\rangle
    = \sum_{i=1}^{n} \langle \Delta x_i^2 \rangle ,
$$

Siden $\langle \Delta x_i \rangle = 0$.

Forventningsverdien til den kvadrerte steglengden er

$$
    \langle \Delta x_i^2 \rangle 
    =  \frac{1}{4} \cdot 1^2 + \frac{1}{4} \cdot (-1)^2 + \frac{1}{2} \cdot 0^2
    = \frac{1}{2} .
$$


Forventningsverdiene blir da

$$
    \langle x_{n}^2 \rangle  = \langle y_{n}^2 \rangle  = \frac{n}{2} .
$$


Forventningsverdien til kvadrert avstand fra startpunkt:

$$
    \langle |\vec{r}_{n}|^2 \rangle  = n .
$$


**Gjennomsnittlig kvadrert avstand fra startpunkt** vil gå mot forventningsverdien for store populasjoner.

In [ ]:
walkers = 1000
N = 50
R = np.zeros((walkers, N, 2))
n_2D = np.arange(N)
for w in range(walkers):
    R[w] = random_walk_2D(N)
    
plt.figure(figsize=(15, 5))
plt.plot(n_2D, n_2D, label="Analytic", color="black")
for w in [10, 100, 1000]:
    dist_squared = np.sum(R[:w]*R[:w], axis=2)
    plt.plot(n_2D, np.mean(dist_squared, axis=0), "--", label=f"{w} walkers")
plt.legend()
plt.xlabel('n')
plt.ylabel(r'$\langle |\vec{r}_{n}|^2 \rangle$')
plt.show()
print()

**Animasjon:** Med mange vandrere kan vi se at de sprer seg som en sirkel med økende radius. 

In [ ]:
video = animate.RandomWalk(R[:400]).video(speed=200)
HTML(video)

### Brownsk bevegelse

Bevegelse til partikler i væske eller gass. Kan simuleres i én, to eller tre dimensjoner.

Tenk deg en **større partikkel som kolliderer med masse små partikler** på veien.

Bevegelsen til den store partikkelen vil da se kaotisk ut.

**Normalfordelt steglengde.** 

Brownsk bevegelse har normalfordelt steglengde rundt 0. 

Steglengden i de forskjellige retningene er uavhengig av hverandre.

In [ ]:
def brownian_motion(N, sigma=1):
    r = np.zeros((N, 2))
    ...
    return r

Forslag til implementasjon er gitt i cellen under.

In [ ]:
def brownian_motion(N, sigma=1):
    r = np.zeros((N, 2))
    dr = np.random.normal(loc=0, scale=sigma, size=(N-1, 2))
    r[1:] = np.cumsum(dr, axis=0)
    return r

**Kaotiske partikler:**

In [ ]:
R = [brownian_motion(200) for _ in range(10)]
video = animate.RandomWalk(R).video(speed=100)
HTML(video)

## Diffusjon

Diffusjonslikningen er gitt ved

$$
    \frac{\partial\rho(\vec{r}, t)}{\partial t} = D \nabla^2 \rho(\vec{r}, t) ,
$$

hvor $\rho$ er partikkeltettheten og $D$ er diffusjonskonstanten.

### Partikkeltetthet i stokastiske simuleringer

Tenk at du simulerer $P$ virrevandrende partikler.

1. Del opp rommet som simuleres i segment, hvor segment $s$ har volum $V_s$.

2. For hvert tidsteg, tell hvor mange partikler $N_s^t$ som befinner seg i segment $s$.

3. Tettheten i hvert segment er da $\rho(\vec{r}_s, t) = N_s^t/V_s$.

**Monte Carlo-simulering av tettheten:** Simuler $P$ virrevandrende partikler $N_\text{MC}$ ganger og bruk gjennomsnittet av alle simuleringene.

### Diffusjon av Brownske partikler i én dimensjon

Diffusjon i én dimensjon er gitt ved

$$
    \frac{\partial \rho(x, t)}{\partial t} = D\frac{\partial^2 \rho(x, t)}{\partial x^2} .
$$

For Brownske partikler er tettheten gitt ved

$$
    \rho(x, t) =  \frac{N}{\sqrt{4\pi D t}} e^{\frac{-x^2}{4 D t}},
$$

hvor $\langle x^2 \rangle = 2 D t$.

**Diffusjonskonstanten:** Vi forenkler og setter  $\Delta t = 1$, slik at $t = n \Delta t = n$.

For virrevandring med $\Delta x = \pm 1$ viste vi at

$$ 
\langle x^2 \rangle = n,
$$

men dette stemmer også for $\Delta x$ med normal distribusjon med $\sigma = 1$.

Vi har da at

$$
    D = \frac{\langle x^2 \rangle}{2 t} = \frac{1}{2} .
$$


**Diffusjon med forenklet Brownsk bevegelse:** Virrevandring med $\Delta x = \pm 1$.

In [ ]:
P = [random_walk_1D(50) for _ in range(10_000)]
video = animate.Diffusion(P, dx=1, D=0.5).video(speed=200)
HTML(video)

**Diffusjon med Brownsk bevegelse:**

In [ ]:
P = [brownian_motion(50)[:,0] for _ in range(10_000)]
video = animate.Diffusion(P, dx=0.25, D=0.5).video(speed=200)
HTML(video)

## Mer avanserte vandrere

Til nå har vi kun sett på ***uavhengige* vandrere**. 

Å simulere $N$ forskjellige vandrere én gang gir da samme resultat som å simulere én vandrer $N$ ganger.

Vandrerne har også kunnet bevege seg helt fritt, med like stor sannsynlighet for å gå i alle retninger. 

### Omgivelser

Dette kan f.eks være
- **Begrense rommet til vandrerne:** Kun la vandrere gå på et gitt område.
- **Virrevandring med bias:** Større sannsynlighet for å gå noen veier enn andre.

### Samspill mellom vandrere

Dette kan f.eks være
- **Vandrere med størrelse:** Vandrere kan ikke oppta samme plass til samme tid.
- **Interaksjoner mellom vandrere:** Som f.eks elektroner som frastøter hverandre.

Slike simuleringer vil avhenge av antall vandrere. For å regne ut tettheten må man derfor gjenta forsøket mange ganger med $P$ vandrere hver gang.

## Oppsummering

**Virrevandring** er å forflytte seg etter formelen

$$
    \vec{r}_{n + 1} = \vec{r}_n +  \Delta \vec{r}_{n+1},
$$

hvor **steglengden $\Delta \vec{r}_{n+1}$** er en **stokastisk variabel**.

**Gjennomsnittlige egenskaper** til virrevandrere vil gå mot **teoretiske forventningsverdier** i takt med **størrelsen til populasjonen**.

**Virrevandrere** kan brukes til å løse **diffusjonslikningen**.

Neste forelesning blir om Markovkjeder.